In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import argparse
import torch
import sys

/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/agostini/miniconda/envs/dles/lib/python3

In [3]:
import tensorflow as tf

In [4]:
args ={}
#args['enc_model_dir']=Path('encoder/saved_models/pretrained.pt')
args['enc_model_dir']=Path('encoder/saved_models/quantized.pt')
args['low_mem']=False

In [5]:
## Print some environment information (for debugging purposes)
print("Running a test of your configuration...\n")
if not torch.cuda.is_available():
    print("Your PyTorch installation is not configured to use CUDA. If you have a GPU ready "
          "for deep learning, ensure that the drivers are properly installed, and that your "
          "CUDA version matches your PyTorch installation. CPU-only inference is currently "
          "not supported.", file=sys.stderr)
    quit(-1)
device_id = torch.cuda.current_device()
gpu_properties = torch.cuda.get_device_properties(device_id)
print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
      "%.1fGb total memory.\n" % 
      (torch.cuda.device_count(),
       device_id,
       gpu_properties.name,
       gpu_properties.major,
       gpu_properties.minor,
       gpu_properties.total_memory / 1e9))

Running a test of your configuration...

Found 4 GPUs available. Using GPU 0 (GeForce RTX 2080 Ti) of compute capability 7.5 with 11.5Gb total memory.



In [6]:
## Load the models one by one.
## Load the models one by one.
print("Preparing the encoder, the synthesizer and the vocoder...")
encoder.load_model(args['enc_model_dir'])

Preparing the encoder, the synthesizer and the vocoder...


RuntimeError: Error(s) in loading state_dict for SpeakerEncoder:
	Missing key(s) in state_dict: "lstm.weight_ih_l0", "lstm.weight_hh_l0", "lstm.bias_ih_l0", "lstm.bias_hh_l0", "lstm.weight_ih_l1", "lstm.weight_hh_l1", "lstm.bias_ih_l1", "lstm.bias_hh_l1", "lstm.weight_ih_l2", "lstm.weight_hh_l2", "lstm.bias_ih_l2", "lstm.bias_hh_l2", "linear.weight", "linear.bias". 
	Unexpected key(s) in state_dict: "linear.scale", "linear.zero_point", "linear._packed_params.weight", "linear._packed_params.bias". 

In [7]:
# Forward an audio waveform of zeroes that lasts 1 second. Notice how we can get the encoder's
# sampling rate, which may differ.
# If you're unfamiliar with digital audio, know that it is encoded as an array of floats 
# (or sometimes integers, but mostly floats in this projects) ranging from -1 to 1.
# The sampling rate is the number of values (samples) recorded per second, it is set to
# 16000 for the encoder. Creating an array of length <sampling_rate> will always correspond 
# to an audio of 1 second.
print("\tTesting the encoder...")
encoder.embed_utterance(np.zeros(encoder.sampling_rate))

	Testing the encoder...


array([0.09639323, 0.        , 0.17957953, 0.00909078, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.13796514,
       0.05528075, 0.        , 0.00052638, 0.04459466, 0.03905624,
       0.15472305, 0.        , 0.00509368, 0.03652696, 0.1459964 ,
       0.02750544, 0.05775689, 0.06767976, 0.04090392, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00556661, 0.        , 0.        , 0.04342674, 0.12426875,
       0.01535764, 0.14615168, 0.        , 0.        , 0.06382234,
       0.        , 0.        , 0.04422446, 0.10181872, 0.12331415,
       0.13299014, 0.0488346 , 0.        , 0.        , 0.        ,
       0.16133443, 0.07981407, 0.1167616 , 0.05399694, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.08142461,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02794344, 0.        , 0.02964833, 0.        , 0.        ,
       0.        , 0.        , 0.00290785, 0.03012798, 0.     

If you were not paying attention, the cell above print thw following parameters:
```
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?)
  device:                   0
  embedding:                (?, ?, 512)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       28.439 Million.
```

In [8]:
%%timeit
# Hit shift enter to run it again
encoder.embed_utterance(np.zeros(encoder.sampling_rate))

5.4 ms ± 4.09 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
